<a href="https://colab.research.google.com/github/RiyaJohn/HE_IndiaMartPriceRangePrediction/blob/master/Phase2_PriceRangePrediction_inprogress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import csv
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import pandas as pd
from scipy import stats
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re
import spacy.cli
spacy.cli.download("en_core_web_lg")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg
nlp = en_core_web_lg.load()

### Data Cleaning


In [113]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab Notebooks/data/Unit Quantity Codes.xlsx"
!ls "/content/drive/My Drive/Colab Notebooks/data/Hackathon_Phase2_sm.xlsx"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/drive/My Drive/Colab Notebooks/data/Unit Quantity Codes.xlsx'
'/content/drive/My Drive/Colab Notebooks/data/Hackathon_Phase2_sm.xlsx'


In [114]:
# Get Static dataset of Indian Customs Quantity Codes and Shipping Measurement Units
qty_codes_file = "/content/drive/My Drive/Colab Notebooks/data/Unit Quantity Codes.xlsx"
qty_codes = pd.read_excel(qty_codes_file)
qty_codes = pd.DataFrame(qty_codes)
qty_codes.head(5)

,Quantity Code,Quantity Description,Quantity Type
0,AMP,AMPULES,NaN
1,BAG,BAGS,NaN
2,BAL,BALE,M
3,BDL,BUNDLES,NaN
4,BKL,BUCKLES,M


In [0]:
# Data collection
data_file = "/content/drive/My Drive/Colab Notebooks/data/Hackathon_Phase2_sm.xlsx"
data = pd.read_excel(data_file, None) # None - Reading all sheets in the file


In [117]:
# Remove NAs and zeros data
for key, value in data.items():
  df = pd.DataFrame(data[key])
  print(df.head(2))
  df = df.dropna() # remove NAs
  df = df[(df != 0).all(1)] # remove rows that have zeros in any of the col.
print(len(df))

         Mcat Name  ... PC_ITEM_MOQ_UNIT_TYPE
0  HMI Touch Panel  ...                 Piece
1  HMI Touch Panel  ...                  Unit

[2 rows x 8 columns]
3331


### Data Processing

In [0]:
# getting the unique set of units in the input data
input_units = []
df['PC_ITEM_MOQ_UNIT_TYPE'] = df['PC_ITEM_MOQ_UNIT_TYPE'].astype(str).str.upper()
input_units.append(df.PC_ITEM_MOQ_UNIT_TYPE.unique().tolist())
input_units = set([item for sublist in input_units for item in sublist])

In [119]:
# Units pre-processing - removing [onwards, per, (s), numbers]
input_units_objs = {}
for unit in input_units:
  input_units_objs[unit] = {}
  
  if re.search('onwards', unit, re.IGNORECASE):
    input_units_objs[unit]["onwards"] = True
#   when 120 per piece given then 120 is price 
  if re.search('[0-9]+( *per)', unit, re.IGNORECASE):
    input_units_objs[unit]["price"] = int(re.search('[0-9]+', re.search('[0-9]+( *per)', unit, re.IGNORECASE).group()).group())
#  removing  [onwards, per, (s), one/1] 
  shorter = re.sub('(onwards *)*([0-9 ]*per *)*(\(s\))*((1|one)(?![\d]))*', '', unit, flags=re.IGNORECASE)
#   removing other numbers to set them as size(could be 5, 1-100 as range or 40' as in inch also) 
  if re.search('[0-9]+([-]?[0-9]+)*', shorter):
    input_units_objs[unit]["size"] = re.search('[0-9]+([-]?[0-9]+)*[\'\"]*', shorter).group()
    shorter = re.sub('[0-9]+([-]?[0-9]+)*[\'\" ]*', '', shorter, flags=re.IGNORECASE)
#    if empty means they were numbers given so set unit to numbers(if 5) else take given unit(if 5 pcs)
  if shorter == "":
    input_units_objs[unit]["shorter"] = "NOS"
  else:
    input_units_objs[unit]["shorter"] = shorter.strip(' /_-.')

print(input_units_objs)
print(len(input_units_objs))

{'GALLON': {'shorter': 'GALLON'}, 'METER': {'shorter': 'METER'}, 'BUNDLE': {'shorter': 'BUNDLE'}, 'NUMBER': {'shorter': 'NUMBER'}, 'PERSON': {'shorter': 'SON'}, 'SET': {'shorter': 'SET'}, 'PER PIECE': {'shorter': 'PIECE'}, '50KG': {'size': '50', 'shorter': 'KG'}, 'BARREL': {'shorter': 'BARREL'}, 'LITER': {'shorter': 'LITER'}, 'METER(S)': {'shorter': 'METER'}, 'BAG': {'shorter': 'BAG'}, '1 BOX': {'shorter': 'BOX'}, 'NOS': {'shorter': 'NOS'}, 'UNIT': {'shorter': 'UNIT'}, 'ROLL': {'shorter': 'ROLL'}, 'KG': {'shorter': 'KG'}, 'MILLILITER': {'shorter': 'MILLILITER'}, 'BOX': {'shorter': 'BOX'}, 'LITRE': {'shorter': 'LITRE'}, 'PER KG': {'shorter': 'KG'}, '1 LTR': {'shorter': 'LTR'}, 'METRIC TON': {'shorter': 'METRIC TON'}, '1 LITER': {'shorter': 'LITER'}, 'BOTTLE': {'shorter': 'BOTTLE'}, 'LONG TON': {'shorter': 'LONG TON'}, 'PIECE': {'shorter': 'PIECE'}, 'PACKET': {'shorter': 'PACKET'}, 'TON': {'shorter': 'TON'}, 'KILOGRAM': {'shorter': 'KILOGRAM'}, 'PACK': {'shorter': 'PACK'}, 'TONS': {'shor

In [120]:
# Getting a matching std. unit code for input units

for input_unit,input_unit_obj in input_units_objs.items():
  unit_code = "" # []
  unit_scr = 100
  for index, row in qty_codes.iterrows():
    shorter_input_unit = input_unit_obj["shorter"] if input_unit_obj["shorter"] != "" else input_unit
    scr1 = nltk.edit_distance(shorter_input_unit, row[0])
    scr2 = nltk.edit_distance(shorter_input_unit, row[1])
    scr = scr1 if scr1 < scr2 else scr2
    if scr < unit_scr:
      unit_code = row[0]
      unit_scr = scr
  if unit_scr <=3:
    input_unit_obj["code"] = unit_code
  else:
    input_unit_obj["code"] = "OTH"

print(input_units_objs)

{'GALLON': {'shorter': 'GALLON', 'code': 'OTH'}, 'METER': {'shorter': 'METER', 'code': 'MTR'}, 'BUNDLE': {'shorter': 'BUNDLE', 'code': 'BDL'}, 'NUMBER': {'shorter': 'NUMBER', 'code': 'NOS'}, 'PERSON': {'shorter': 'SON', 'code': 'TON'}, 'SET': {'shorter': 'SET', 'code': 'SET'}, 'PER PIECE': {'shorter': 'PIECE', 'code': 'PCS'}, '50KG': {'size': '50', 'shorter': 'KG', 'code': 'KGA'}, 'BARREL': {'shorter': 'BARREL', 'code': 'BAL'}, 'LITER': {'shorter': 'LITER', 'code': 'LTR'}, 'METER(S)': {'shorter': 'METER', 'code': 'MTR'}, 'BAG': {'shorter': 'BAG', 'code': 'BAG'}, '1 BOX': {'shorter': 'BOX', 'code': 'BOX'}, 'NOS': {'shorter': 'NOS', 'code': 'NOS'}, 'UNIT': {'shorter': 'UNIT', 'code': 'UNT'}, 'ROLL': {'shorter': 'ROLL', 'code': 'ROL'}, 'KG': {'shorter': 'KG', 'code': 'KGA'}, 'MILLILITER': {'shorter': 'MILLILITER', 'code': 'MLT'}, 'BOX': {'shorter': 'BOX', 'code': 'BOX'}, 'LITRE': {'shorter': 'LITRE', 'code': 'LTR'}, 'PER KG': {'shorter': 'KG', 'code': 'KGA'}, '1 LTR': {'shorter': 'LTR', '

In [0]:
def onwards_addon_price(price, percentage_to_add):
  new_price = price+(price*percentage_to_add)/100
  return new_price

def find_priority(cnt, total_cnt):
  if cnt > total_cnt/2:
    priority_scr = 1
  elif cnt > total_cnt/3:
    priority_scr = 2
  elif cnt > total_cnt/5:
    priority_scr = 3
  elif cnt > total_cnt/10:
    priority_scr = 4
  else:
    priority_scr = 5
  return priority_scr

def find_modes(data, n):
  modes = data.value_counts().head(n*4)
  n_modes = {}
  for val, cnt in modes.items():
    if cnt in n_modes:
      n_modes[cnt].append(val)
    else:
      n_modes[cnt] = [val]
    if len(n_modes) == n:
      break
  while len(n_modes) < 3:
    n_modes[-1*len(n_modes)] = []
    
  return list(n_modes.values())

In [122]:
# Update price col. if onwards true or price info given 
# Update desc. col(s) with size info TODO
for index, row in df.iterrows(): 
  if "onwards" in input_units_objs[row['PC_ITEM_MOQ_UNIT_TYPE']]:
    row['PC_ITEM_FOB_PRICE'] = onwards_addon_price(row['PC_ITEM_FOB_PRICE'], 10)
  if "size" in input_units_objs[row['PC_ITEM_MOQ_UNIT_TYPE']]:
    row['FK_IM_SPEC_MASTER_DESC'] = str(row['FK_IM_SPEC_MASTER_DESC'])+", size"
    row['FK_IM_SPEC_OPTIONS_DESC'] = str(row['FK_IM_SPEC_OPTIONS_DESC'])+", "+str(input_units_objs[row['PC_ITEM_MOQ_UNIT_TYPE']]['size'])
#   Removing (inch), (m) etc in isq column and placing it in isq options 
  if re.search('\(.*\)', str(row['FK_IM_SPEC_MASTER_DESC'])): 
    row['FK_IM_SPEC_OPTIONS_DESC'] = str(row['FK_IM_SPEC_OPTIONS_DESC'])+re.search('\(.*\)', str(row['FK_IM_SPEC_MASTER_DESC'])).group(0)
    row['FK_IM_SPEC_MASTER_DESC'] = re.sub('\(.*\)', "", str(row['FK_IM_SPEC_MASTER_DESC']))
# Replacing input units with matching std. unit code/s
df['PC_ITEM_MOQ_UNIT_TYPE'] = df['PC_ITEM_MOQ_UNIT_TYPE'].map(lambda unit : input_units_objs[unit]["code"]) 
  
print(df.head(10)) 

         Mcat Name  ... PC_ITEM_MOQ_UNIT_TYPE
0  HMI Touch Panel  ...                   PCS
1  HMI Touch Panel  ...                   UNT
2  HMI Touch Panel  ...                   UNT
3  HMI Touch Panel  ...                   UNT
4  HMI Touch Panel  ...                   UNT
5  HMI Touch Panel  ...                   UNT
6  HMI Touch Panel  ...                   UNT
7  HMI Touch Panel  ...                   UNT
8  HMI Touch Panel  ...                   UNT
9  HMI Touch Panel  ...                   UNT

[10 rows x 8 columns]


In [0]:
df['FK_IM_SPEC_MASTER_DESC'] = df['FK_IM_SPEC_MASTER_DESC'].map(lambda x: x.lower())
df['FK_IM_SPEC_MASTER_DESC'] = df['FK_IM_SPEC_MASTER_DESC'].map(lambda x: re.sub('ing', '', x))

In [0]:
cond_col = df[['FK_IM_SPEC_MASTER_DESC']]
df = df[cond_col.replace(cond_col.apply(pd.Series.value_counts)).gt(5).all(1)]
print(df.head(10)) 

In [125]:
df_by_mcat = df.groupby('Mcat Name')
mcat_sim_isqs = {}  
for mcat_name, mcat_group in df_by_mcat:
  isq_grouped_cnts = mcat_group.FK_IM_SPEC_MASTER_DESC.value_counts()
  similar_isqs = []
  for isq1 in isq_grouped_cnts.keys():
    for isq2 in isq_grouped_cnts.keys():
      if isq1 != isq2 and any(isq1 in similar_isq for similar_isq in similar_isqs) == False and nlp(isq1).similarity(nlp(isq2)) > 0.8:
        appended = False
        for similar_isq in similar_isqs:
          if isq2 in similar_isq:
            appended = True
            similar_isq.append(isq1)  
        if appended == False:
          similar_isqs.append([isq1, isq2])
  print(similar_isqs) 
      
  sim_isqs = {}    
  for similar_isq in similar_isqs:
    key = max(dict((key,isq_grouped_cnts[key]) for key in similar_isq if key in isq_grouped_cnts), key=isq_grouped_cnts.get)
    sim_isqs[key] = similar_isq
  mcat_sim_isqs[mcat_name] = sim_isqs


[['product type', 'packag type', 'type'], ['pack size', 'packag size'], ['grade standard', 'grade']]
[['display type', 'type', 'product type', 'screen type'], ['screen size', 'display size', 'display size (inch)', 'size']]
[['type', 'product type']]
[['number of core', 'number of cores'], ['type', 'packag type'], ['material', 'core material', 'insulat material']]


In [0]:
for index, row in df.iterrows(): 
  sim_isqs_by_mcat = mcat_sim_isqs[row["Mcat Name"]]
  if row["FK_IM_SPEC_MASTER_DESC"] in sim_isqs_by_mcat.values():
    row["FK_IM_SPEC_MASTER_DESC"] = list(sim_isqs_by_mcat.keys())[list(sim_isqs_by_mcat.values()).index(row["FK_IM_SPEC_MASTER_DESC"])]

In [0]:
df['FK_IM_SPEC_OPTIONS_DESC'] = df['FK_IM_SPEC_OPTIONS_DESC'].astype(str)

### Statistics

In [131]:
# res = {}
df_by_mcat = df.groupby('Mcat Name')
with open('output_unit_wise.csv', 'w') as writeFile:
  writer = csv.writer(writeFile)
  writer.writerows([["Mcat Name", "PC_ITEM_MOQ_UNIT_TYPE", "Unit wise Min. Price", "Unit wise Max. Price", "Unit Priority", "Most Common Price 1", "Most Common Price 2", "Most Common Price 3"]])
  
  for mcat_name, mcat_group in df_by_mcat:
    res[mcat_name] = {}
    unit_grouped_df = mcat_group.groupby('PC_ITEM_MOQ_UNIT_TYPE')
    for unit, group in unit_grouped_df: 
      group = group[(np.abs(stats.zscore(group.PC_ITEM_FOB_PRICE)) < 5)] # Remove rows with outliers(+/-3 std. deviation away) in Price column
      n_modes = find_modes(group.PC_ITEM_FOB_PRICE, 3)

      if len(group) > 3:
        priority_scr = find_priority(len(group), len(mcat_group))
        skew_score = group.PC_ITEM_FOB_PRICE.skew()
        if skew_score > 1 or skew_score < -1:
          log_data = np.log(group.PC_ITEM_FOB_PRICE)
          price_range = log_data.quantile([0.1,1])
#           res[mcat_name][unit] = (round(np.exp(price_range[0.1])), round(np.exp(price_range[1])), priority_scr, n_modes)
          writer.writerows([[mcat_name, unit, round(np.exp(price_range[0.1])), round(np.exp(price_range[1])), priority_scr, n_modes[0], n_modes[1], n_modes[2]]])
        else:
          price_range = group.PC_ITEM_FOB_PRICE.quantile([0.1,1])
#           res[mcat_name][unit] = (round(price_range[0.1]), round(price_range[1]), priority_scr, n_modes)
          writer.writerows([[mcat_name, unit, round(price_range[0.1]), round(price_range[1]), priority_scr, n_modes[0], n_modes[1], n_modes[2]]])

          
writeFile.close() 
!cp output_unit_wise.csv drive/My\ Drive/

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2315: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in less
  # Remove the CWD from sys.path while we load stuff.


In [133]:
# res = {}
df_by_mcat = df.groupby('Mcat Name')
with open('output_unit_isq_wise.csv', 'w') as writeFile:
  writer = csv.writer(writeFile)
  writer.writerows([["Mcat Name", "PC_ITEM_MOQ_UNIT_TYPE", "FK_IM_SPEC_MASTER_DESC", "FK_IM_SPEC_OPTIONS_DESC", "ISQ Min. Price", "ISQ Max. Price", "ISQ Priority"]])
  for mcat_name, mcat_group in df_by_mcat:
#     res[mcat_name] = {}
    unit_isq_grouped_df = mcat_group.groupby(['PC_ITEM_MOQ_UNIT_TYPE','FK_IM_SPEC_MASTER_DESC', 'FK_IM_SPEC_OPTIONS_DESC'])
    for isq, group in unit_isq_grouped_df: 
      if len(group) > 5:
        group = group[(np.abs(stats.zscore(group.PC_ITEM_FOB_PRICE)) < 5)] # Remove rows with outliers(+/-5 std. deviation away) in Price column

        if len(group) > 3:
          priority_scr = find_priority(len(group), len(mcat_group))
          skew_score = group.PC_ITEM_FOB_PRICE.skew()
          if skew_score > 1 or skew_score < -1:
            log_data = np.log(group.PC_ITEM_FOB_PRICE)
            price_range = log_data.quantile([0.1,1])
#             res[mcat_name][isq] = (round(np.exp(price_range[0.1])), round(np.exp(price_range[1])), priority_scr)
            writer.writerows([[mcat_name, isq[0], isq[1], isq[2], round(np.exp(price_range[0.1])), round(np.exp(price_range[1])), priority_scr ]])
          else:
            price_range = group.PC_ITEM_FOB_PRICE.quantile([0.1,1])
#             res[mcat_name][isq] = (round(price_range[0.1]), round(price_range[1]), priority_scr)
            writer.writerows([[mcat_name, isq[0], isq[1], isq[2], round(price_range[0.1]), round(price_range[1]), priority_scr ]])
          

writeFile.close()   
!cp output_unit_isq_wise.csv drive/My\ Drive/

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2315: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in less
  # Remove the CWD from sys.path while we load stuff.
